In [1]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn. metrics import accuracy_score,confusion_matrix


In [2]:
# Cambiar directorio
os.chdir("..")

In [3]:
# Cargar los datos que tenemos disponibles
#data = pd.read_csv("data/churn.csv")
data = pd.read_csv("data/churn.csv")


In [4]:
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Nos deshacemos de las columnas que no contribuyen en mucho
data = data.drop(data.columns[0:3], axis=1)

In [6]:
# eliminamos los datos nulos
for i, column in enumerate(list([str(d) for d in data.dtypes])):
    if column == "object":
        data[data.columns[i]] = data[data.columns[i]].fillna(data[data.columns[i]].mode())
    else:
        data[data.columns[i]] = data[data.columns[i]].fillna(data[data.columns[i]].median())

In [7]:
# Convertimos los datos en formato categorico
column_equivalence = {}
features = list(data.columns)
for i in data.select_dtypes(include='object').columns:
  le = LabelEncoder()
  data[i]= le.fit_transform(data[i])
  clase = le.classes_
  current_column_equivalence = dict(enumerate(clase))
  column_equivalence[i] = dict((v,k) for k,v in current_column_equivalence.items())
  

In [8]:
# Generar los datos para poder separar la variable de respuesta de los datos que tenemos disponibles
X = data
y = X.pop(data.columns[-1])

In [9]:
# Separar los datos en datos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [10]:
# # el modelo que mas se ajunto fue un clacificador por Gradien Boosting con los siguientes parametros
# Crear el modelo y entrenarlo
clf_gb =  GradientBoostingClassifier(n_estimators=400,max_depth=2,learning_rate=.2).fit(X, y)

In [11]:
# Medir los resultados obtenidos
print('Confusion Matrix')
print(confusion_matrix(y_test, clf_gb.predict(X_test)))
print('_'*15)
print('ACCURACY: ',accuracy_score(y_test, clf_gb.predict(X_test)))

Confusion Matrix
[[2580   77]
 [ 288  355]]
_______________
ACCURACY:  0.8893939393939394


In [12]:
# Generar el binario del modelo para reutilizarlo, equivalencia de variables categoricas y caracteristicas del modelo
import pickle
pickle.dump(clf_gb, open("churn/models/model.pk", "wb"))
pickle.dump(column_equivalence, open("churn/models/column_equivalence.pk", "wb"))
pickle.dump(features, open("churn/models/features.pk", "wb"))